In [10]:
import polars as pl
import numpy as np
from sklearn import tree
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv("dataset.csv")
df = df.drop(df.columns[0], axis=1)
#df = df.drop(df.columns[2], axis=1) #Enlever la description car ca devient le seul critere de l'arbre
df = df.drop(df.columns[5], axis=1) #Enlever la date (car trop d'options), remettre une fois qu'on aura mis au propre
x_features = df.columns.delete(1)
df.head()

,community_area,primary_type,Arrest,Domestic,location_description,Comm_Area,Hardship_Score,Crowded_Housing_Per,Below_Poverty_Household_Income,Unemployment_Rate,No_High_School_Diploma,Under_18_over_64,Per_Capita_Income
0,63,OFFENSE AND KIDNAPPING,False,True,APARTMENT,Gage Park,65.3,12.1,18.5,13.7,22.4,39.0,13605.0
1,63,OFFENSE AND KIDNAPPING,False,True,APARTMENT,Gage Park,65.3,12.1,18.5,13.7,22.4,39.0,13605.0
2,30,OFFENSE AND KIDNAPPING,False,True,RESIDENCE,South Lawndale,70.6,13.6,30.3,8.7,28.8,35.3,11345.0
3,46,PUBLIC ORDER CRIMES,False,False,APARTMENT,South Chicago,54.9,3.4,28.9,20.3,11.6,41.1,16996.0
4,16,THEFT,False,False,RESIDENCE,Irving Park,32.3,3.5,10.3,5.8,10.3,30.2,31043.0


In [ ]:
df['']

In [3]:
X = df.drop('primary_type', axis=1)
y = df['primary_type']

In [4]:
primary = LabelEncoder()
y_encoded = primary.fit_transform(y)

y_encoded

array([2, 2, 2, ..., 3, 5, 3])

In [5]:
onehot_encoder = OneHotEncoder()
column_transformer = ColumnTransformer(
    transformers=[
        ('cat', onehot_encoder, ['location_description', 'Comm_Area'])
    ],
    remainder='passthrough' 
)
X_encoded = column_transformer.fit_transform(X)
all_encoded_feature_names = column_transformer.get_feature_names_out()
encoded_feature_names = column_transformer.named_transformers_['cat'].get_feature_names_out(['location_description', 'Comm_Area'])
X_encoded_df = pd.DataFrame(X_encoded.toarray(), columns=all_encoded_feature_names)
X_encoded_df.to_csv('X_OneHotEncoder.csv', index=False)
X_encoded_df

,cat__location_description_ABANDONED BUILDING,cat__location_description_AIRCRAFT,cat__location_description_AIRPORT BUILDING NON-TERMINAL - NON-SECURE AREA,cat__location_description_AIRPORT BUILDING NON-TERMINAL - SECURE AREA,cat__location_description_AIRPORT EXTERIOR - NON-SECURE AREA,cat__location_description_AIRPORT EXTERIOR - SECURE AREA,cat__location_description_AIRPORT PARKING LOT,cat__location_description_AIRPORT TERMINAL LOWER LEVEL - NON-SECURE AREA,cat__location_description_AIRPORT TERMINAL LOWER LEVEL - SECURE AREA,cat__location_description_AIRPORT TERMINAL MEZZANINE - NON-SECURE AREA,...,remainder__community_area,remainder__Arrest,remainder__Domestic,remainder__Hardship_Score,remainder__Crowded_Housing_Per,remainder__Below_Poverty_Household_Income,remainder__Unemployment_Rate,remainder__No_High_School_Diploma,remainder__Under_18_over_64,remainder__Per_Capita_Income
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,63.0,0.0,1.0,65.3,12.1,18.5,13.7,22.4,39.0,13605.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,63.0,0.0,1.0,65.3,12.1,18.5,13.7,22.4,39.0,13605.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,30.0,0.0,1.0,70.6,13.6,30.3,8.7,28.8,35.3,11345.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,46.0,0.0,0.0,54.9,3.4,28.9,20.3,11.6,41.1,16996.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,16.0,0.0,0.0,32.3,3.5,10.3,5.8,10.3,30.2,31043.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045246,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,68.0,0.0,0.0,70.5,4.0,42.4,34.1,16.1,41.5,12777.0
1045247,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,49.0,0.0,0.0,52.6,2.2,24.0,24.3,9.6,41.8,19755.0
1045248,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,48.0,0.0,0.0,38.4,2.0,13.7,12.3,8.1,37.8,27488.0
1045249,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,44.0,0.0,1.0,47.9,2.9,26.1,19.0,8.0,37.9,21416.0


In [6]:
x_array = X_encoded_df.to_numpy()
x_array

array([[0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 2.2400e+01, 3.9000e+01,
        1.3605e+04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 2.2400e+01, 3.9000e+01,
        1.3605e+04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 2.8800e+01, 3.5300e+01,
        1.1345e+04],
       ...,
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 8.1000e+00, 3.7800e+01,
        2.7488e+04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 8.0000e+00, 3.7900e+01,
        2.1416e+04],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, ..., 8.2000e+00, 3.6700e+01,
        1.9117e+04]])

In [7]:
X_train, X_test, y_train, y_test = train_test_split(x_array, y_encoded, test_size = 0.3) 

model = MLPClassifier(max_iter=2000, hidden_layer_sizes=10, learning_rate="constant", learning_rate_init=0.001)
model.fit(X_train, y_train)

y_valid=model.predict(X_train)
y_pred=model.predict(X_test)

print("Learning errors")
print("    Accurary = ", accuracy_score(y_train, y_valid))

print("\nPrediction errors")
print("    Accurary = ", accuracy_score(y_test, y_pred))

print("\nWhat is inside :")
print(" Number of iterrations :", model.n_iter_)
print(" Number of layers : ", model.n_layers_)
print(" Coefs : ", model.coefs_)
print(" Tresholds :", model.intercepts_)


print("\n** Learning performances: **") 
# make predictions
y_valid = model.predict(X_train)
#evaluate results
print("Confusion Matrix: ") 
print(confusion_matrix(y_train, y_valid))
print ("Accuracy : ", accuracy_score(y_train, y_valid)*100) 
print("** Prediction performances: **") 
# make predictions
y_pred = model.predict(X_test)
#evaluate results
print("Confusion Matrix: ") 
print(confusion_matrix(y_test, y_pred))
print ("Accuracy : ", accuracy_score(y_test,y_pred)*100) 

Learning errors
    Accurary =  0.5730522431407388

Prediction errors
    Accurary =  0.572559762226701

What is inside :
 Number of iterrations : 94
 Number of layers :  3
 Coefs :  [array([[ 1.41774646e-315, -3.64329825e+000,  4.99116308e+000, ...,
        -2.91313424e-315,  1.65209708e-315,  3.94628413e-315],
       [-1.51663775e-315, -3.60382744e+000,  1.84682165e+000, ...,
        -3.74996984e-315,  1.02387507e-315,  3.51944125e-315],
       [-1.42031329e-316, -2.28329930e+000,  4.33439586e+000, ...,
         4.06561848e-315, -1.26328184e-316,  5.43625588e-316],
       ...,
       [-2.58795441e-315,  1.25967985e-001,  3.30520204e-001, ...,
        -1.93676086e-315,  4.01965671e-315,  8.98260904e-316],
       [-3.34484728e-315,  4.04978255e-001,  4.57773882e-001, ...,
        -1.87092962e-315, -5.08715680e-317, -2.96635286e-315],
       [-3.83587493e-315, -7.07691317e-005,  8.35201229e-004, ...,
         3.55854355e-315, -3.38693957e-315, -1.11301071e-315]]), array([[-3.63992182e-3

In [9]:
model = MLPClassifier(max_iter=2000, hidden_layer_sizes=10, learning_rate="constant", learning_rate_init=0.001)
model.fit(X_train, y_train)

y_valid=model.predict(X_train)
y_pred=model.predict(X_test)

print("Learning errors")
print("    Accurary = ", accuracy_score(y_train, y_valid))

print("\nPrediction errors")
print("    Accurary = ", accuracy_score(y_test, y_pred))

print("\nWhat is inside :")
print(" Number of iterrations :", model.n_iter_)
print(" Number of layers : ", model.n_layers_)
print(" Coefs : ", model.coefs_)
print(" Tresholds :", model.intercepts_)


print("\n** Learning performances: **") 
# make predictions
y_valid = model.predict(X_train)
#evaluate results
print("Confusion Matrix: ") 
print(confusion_matrix(y_train, y_valid))
print ("Accuracy : ", accuracy_score(y_train, y_valid)*100) 
print("** Prediction performances: **") 
# make predictions
y_pred = model.predict(X_test)
#evaluate results
print("Confusion Matrix: ") 
print(confusion_matrix(y_test, y_pred))
print ("Accuracy : ", accuracy_score(y_test,y_pred)*100) 

Learning errors
    Accurary =  0.4313035159052858

Prediction errors
    Accurary =  0.4314871036048677

What is inside :
 Number of iterrations : 15
 Number of layers :  3
 Coefs :  [array([[ 1.00757227e-315, -5.43187317e-316,  2.22829607e-315, ...,
         3.56616539e-316, -1.69487731e-315,  1.96550223e-316],
       [-4.20188168e-315,  3.65424076e-315, -1.92502467e-315, ...,
        -7.48928475e-316, -5.87592609e-316,  3.78450237e-315],
       [-1.39358946e-315,  1.73848154e-315,  1.64322775e-315, ...,
         2.71852466e-315,  1.27122712e-315,  6.24254033e-316],
       ...,
       [ 2.99929529e-315, -4.68006152e-166, -1.95815498e-002, ...,
        -2.65716805e-316, -9.30722840e-316,  1.55725146e-315],
       [-3.07600802e-315, -2.03962616e-099, -2.59712077e-002, ...,
         2.36323031e-316,  3.17607206e-315,  3.08112123e-316],
       [-1.93321292e-315, -2.19129155e-002, -2.85977366e-002, ...,
         6.35272107e-316,  2.66730932e-078,  6.24204261e-316]]), array([[ 2.76377093e-